In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline

In [7]:
import tensorflow.keras as tk
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd

In [68]:
# def y2indicator(Y):
#     N = len(Y)
#     K = len(set(Y))
#     I = np.zeros((N, K))
#     I[np.arange(N), Y] = 1
#     return I

In [96]:
#Y = y_train[0:15]

In [92]:
#Y = y2indicator(y_train)
# I = np.zeros((15,10))
# I[np.arange(15),Y] = 1

In [94]:
#I

In [95]:
#Y

In [9]:
df_mnist_train = pd.read_csv('../large_files/fashionmnist/fashion-mnist_train.csv')
df_mnist_test = pd.read_csv('../large_files/fashionmnist/fashion-mnist_test.csv')

In [11]:
df_mnist_train.head(2)
df_mnist_test.head(2)

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,9,8,...,103,87,56,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,34,0,0,0,0,0,0,0,0,0


In [15]:
df_mnist_train.shape
df_mnist_train.info()

(60000, 785)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Columns: 785 entries, label to pixel784
dtypes: int64(785)
memory usage: 359.3 MB


In [20]:
y_train = df_mnist_train['label']
x_train = df_mnist_train.iloc[:,1:]

y_test = df_mnist_test['label']
x_test = df_mnist_test.iloc[:,1:]

In [28]:
y_train.nunique()

10

In [29]:
## one hot encoding
from tensorflow.keras.utils import to_categorical
y_cat_train = to_categorical(y_train,10)
y_cat_test = to_categorical(y_test,10)

In [31]:
y_cat_train[0:2,:]

array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

In [32]:
## Standardizing the values
from sklearn.preprocessing import MinMaxScaler
scalar_object = MinMaxScaler()

In [33]:
scalar_object.fit(x_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [34]:
scaled_x_train = scalar_object.transform(x_train)
scaled_x_test = scalar_object.transform(x_test)

In [55]:
# scaled_x_train
# scaled_x_test

In [41]:
scaled_x_train[0,0:20]
scaled_x_train.max()

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

1.0

### Reshaping for CNN

In [56]:
scaled_x_train.shape
X_train = scaled_x_train.reshape(scaled_x_train.shape[0],28,28,1)
X_train.shape

scaled_x_test.shape
X_test = scaled_x_test.reshape(scaled_x_test.shape[0],28,28,1)
X_test.shape

(60000, 784)

(60000, 28, 28, 1)

(10000, 784)

(10000, 28, 28, 1)

### Model Building

In [42]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Flatten

In [60]:
model = Sequential()

model.add(Conv2D(filters=32,kernel_size=(3,3),input_shape=(28,28,1),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(Dense(10,activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

In [61]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 5408)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               692352    
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
Total params: 693,962
Trainable params: 693,962
Non-trainable params: 0
_________________________________________________________________


In [97]:
r = model.fit(X_train,y_cat_train,epochs=2)

Epoch 1/2
60000/60000 [==============================] - 90s 1ms/step - loss: 0.2303 - acc: 0.9174
Epoch 2/2
60000/60000 [==============================] - 111s 2ms/step - loss: 0.2047 - acc: 0.9271


In [65]:
model.evaluate(X_test,y_cat_test)

10000/10000 [==============================] - 3s 317us/step


[0.2600961853027344, 0.9073]

In [63]:
from sklearn.metrics import classification_report

In [64]:
predict_classes = model.predict_classes(X_test)

In [67]:
print(classification_report(y_test,predict_classes))

             precision    recall  f1-score   support

          0       0.82      0.90      0.86      1000
          1       0.98      0.99      0.98      1000
          2       0.84      0.88      0.86      1000
          3       0.95      0.89      0.92      1000
          4       0.89      0.82      0.86      1000
          5       0.94      0.98      0.96      1000
          6       0.76      0.75      0.75      1000
          7       0.97      0.91      0.94      1000
          8       0.98      0.97      0.98      1000
          9       0.95      0.97      0.96      1000

avg / total       0.91      0.91      0.91     10000

